In [37]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import json
from PIL import Image
import os
from torch.utils.data import random_split
import torch.nn as nn
import torch.optim as optim
import torchsummary
import torch.nn.init as init

In [2]:
# Download data in json format

os.makedirs("images_train", exist_ok=True)
os.makedirs("images_test", exist_ok=True)


train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)


test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

# Create JSON files for training and test datasets
def create_json(dataset, json_filename):
    data_list = []
    for i, (image, label) in enumerate(dataset):
        data_list.append({"img_path": f"{i}.png", "labels": label})
        base_name = os.path.basename(json_filename)
        type_data = base_name.split('_')[0] # returns train or test
        directory = "images_" + type_data
        image_path = os.path.join(directory, f"{i}.png")
        torchvision.utils.save_image(image, image_path)
    with open(json_filename, 'w') as json_file:
      for data in data_list:
        json.dump(data, json_file)
        json_file.write('\n')

create_json(train_data, "train_data.json")
create_json(test_data, "test_data.json")

100%|██████████| 26421880/26421880 [00:02<00:00, 10759030.94it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 201313.11it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3836116.01it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 13957515.83it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [47]:
# Dataset customed

class CustomImageDataset(Dataset):

    def __init__(self, json_filename, transform=None):
        """
        Arguments:
            json_filename (string): Json file with images paths and labels.
            transform (callable, optional): Optional transform to be applied.
        """
        self.data_list = []
        with open(json_filename, 'r') as file:
          for line in file:
              loaded_dict = json.loads(line)
              self.data_list.append(loaded_dict)
        self.transform = transform
        self.json_filename = json_filename

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
      img_path_aux = self.data_list[idx]['img_path']
      base_name = os.path.basename(self.json_filename)
      type_data = base_name.split('_')[0]
      directory = "images_" + type_data
      img_path = os.path.join(directory, img_path_aux)
      image = Image.open(img_path)
      if self.transform:
            image = self.transform(image)

      label = self.data_list[idx]['labels']
      return image, label


# Transforming the image in getitem
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    #transforms.RandomCrop((224,224))
    transforms.Grayscale(num_output_channels=1), # 28x28x1
    transforms.ToTensor()
    #transforms.ToTensor(),
    #transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = CustomImageDataset("train_data.json", transform)
test_dataset = CustomImageDataset("test_data.json", transform)

In [48]:
# This code divides the training in train-val data

# 70 % of 60.000 training and 30% validation
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

print("Train size: ", train_size, "\nValidation size: ", val_size, "\nTest size: ", len(test_dataset))
new_train_dataset, new_val_dataset = random_split(train_dataset, [train_size, val_size])

batch_size = 32
train_loader = DataLoader(new_train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(new_val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

Train size:  48000 
Validation size:  12000 
Test size:  10000


In [55]:
# BASELINE: LeNet-5 architecture
class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature = nn.Sequential(
            # convolution: 6 out channels/filters = 6@28x28, a convolution of 5x5 is applied -> kernel_size 5
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2),   # padding 2 because input images are 28x28 (not 32x32)
            nn.ReLU(),
            # average pooling
            nn.AvgPool2d(kernel_size=2, stride=2),  # 14*14
            # convolution: it recieves 6 from previous convolution and now 16@10x10
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),

        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=16*5*5, out_features=120),
            nn.ReLU(),
            nn.Linear(in_features=120, out_features=84),
            nn.ReLU(),
            nn.Linear(in_features=84, out_features=10),
        )

    def forward(self, x):
      return self.classifier(self.feature(x))


# Kaiming Uniform initialization
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        init.kaiming_uniform_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            init.constant_(m.bias, 0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # in Alex computer no cuda, already checked, Alimo you could check

baseline_model = LeNet5().to(device)
baseline_model.apply(init_weights)
optimizer = optim.Adam(baseline_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

torchsummary.summary(baseline_model, input_size=(1,28 , 28))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
              ReLU-2            [-1, 6, 28, 28]               0
         AvgPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 16, 10, 10]           2,416
              ReLU-5           [-1, 16, 10, 10]               0
         AvgPool2d-6             [-1, 16, 5, 5]               0
           Flatten-7                  [-1, 400]               0
            Linear-8                  [-1, 120]          48,120
              ReLU-9                  [-1, 120]               0
           Linear-10                   [-1, 84]          10,164
             ReLU-11                   [-1, 84]               0
           Linear-12                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
---------------------------------

In [52]:
def train(model, device, train_loader, optimizer, criterion, epochs = 5):
    for epoch in range(epochs):
        running_loss = 0.0
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            #print(images.shape)
            # Forward Pass
            output = model(images)
            loss = criterion(output, labels)

            # Backward Pass
            loss.backward()
            optimizer.step() # updates model parameters using gradient computings by back propagation and applies the optimization algo

            running_loss += loss.item()
            if batch_idx % 100 == 99:
                print(f'Epoch {epoch+1}, Batch {batch_idx+1}/{len(train_loader)}, Loss: {running_loss/100:.4f}')
                running_loss = 0.0


In [ ]:
train(baseline_model, device, train_loader, optimizer, criterion)


Epoch 1, Batch 100/1500, Loss: 1.1154
Epoch 1, Batch 200/1500, Loss: 0.6271
